# Способы задания GPU в Tensorflow

Есть как минимум три способа задать GPU:
<ul>
 <li>Через контекст with tf.device(device_name) </li>
 <li>Через конфиг сессии</li>
 <li>Через переменную окружения CUDA_VISIBLE_DEVICES</li>
</ul>

Если в данном ноутбуке запускать последовательно несколько ячеек с кодом, то они могут падать с ошибкой. Для теста выберете одну ячейку. В сессии передаётся параметр log_device_placement=True для того, чтобы в консоль выводилась информация о привязке операций к устройствам.

## 1. Через контекст with tf.device(device_name)

Данный способ позволяет выбрать конкретные устройства для создаваемых операций.

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/cpu:0'): # имя устройства
        a = tf.constant(3.0, name='a')
    with tf.device('/gpu:0'): # имя устройства
        b = tf.constant(4.0, name='b')
    c = a + b
    print(sess.run([a, b, c]))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 537567154419360896
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3180108185
locality {
  bus_id: 1
}
incarnation: 4721743360676997290
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
[3.0, 4.0, 7.0]


#### Вывод консоли хаба:

In [ ]:
# [name: "/device:CPU:0"
# device_type: "CPU"
# memory_limit: 268435456
# locality {
# }
# incarnation: 3069901166767118002
# , name: "/device:GPU:0"
# device_type: "GPU"
# memory_limit: 7969613415
# locality {
#   bus_id: 1
# }
# incarnation: 18230686911253752097
# physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:02:00.0, compute capability: 6.1"
# , name: "/device:GPU:1"
# device_type: "GPU"
# memory_limit: 224526336
# locality {
#   bus_id: 1
# }
# incarnation: 6531455301848960921
# physical_device_desc: "device: 1, name: GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
# ]
# .............
# add: (Add): /job:localhost/replica:0/task:0/device:GPU:0
# 2018-01-31 15:01:11.645857: I tensorflow/core/common_runtime/placer.cc:874] add: (Add)/job:localhost/replica:0/task:0/device:GPU:0
# b: (Const): /job:localhost/replica:0/task:0/device:GPU:0
# 2018-01-31 15:01:11.645898: I tensorflow/core/common_runtime/placer.cc:874] b: (Const)/job:localhost/replica:0/task:0/device:GPU:0
# a: (Const): /job:localhost/replica:0/task:0/device:CPU:0
# 2018-01-31 15:01:11.645914: I tensorflow/core/common_runtime/placer.cc:874] a: (Const)/job:localhost/replica:0/task:0/device:CPU:0
# [3.0, 4.0, 7.0]

## 2. Через конфиг сессии

Через параметр device_count можно задать <b> количество </b> видимых устройств.

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

device_count = {
    'CPU' : 1, # сделать доступным один процессор
    'GPU' : 0  # ни одна GPU не видна
}

with tf.Session(config=tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 0}, log_device_placement=True)) as sess1:
    a = tf.constant(3.0, name='a1')
    b = tf.constant(4.0, name='b1') 
    print(sess1.run([a, b]))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5300259258099670425
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3180108185
locality {
  bus_id: 1
}
incarnation: 9538397050493116081
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
[3.0, 4.0]


#### Вывод консоли хаба:

In [4]:
# [name: "/device:CPU:0"
# device_type: "CPU"
# memory_limit: 268435456
# locality {
# }
# incarnation: 8456035870316386041
# , name: "/device:GPU:0"
# device_type: "GPU"
# memory_limit: 7969613415
# locality {
#   bus_id: 1
# }
# incarnation: 17759846440389336139
# physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:02:00.0, compute capability: 6.1"
# , name: "/device:GPU:1"
# device_type: "GPU"
# memory_limit: 224526336
# locality {
#   bus_id: 1
# }
# incarnation: 424212233639155799
# physical_device_desc: "device: 1, name: GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
# ]
# ..........
# b1: (Const): /job:localhost/replica:0/task:0/device:CPU:0
# 2018-01-31 15:02:25.748722: I tensorflow/core/common_runtime/placer.cc:874] b1: (Const)/job:localhost/replica:0/task:0/device:CPU:0
# a1: (Const): /job:localhost/replica:0/task:0/device:CPU:0
# 2018-01-31 15:02:25.748760: I tensorflow/core/common_runtime/placer.cc:874] a1: (Const)/job:localhost/replica:0/task:0/device:CPU:0
# [3.0, 4.0]

## 3. Через переменную окружения CUDA_VISIBLE_DEVICES

Через переменную CUDA_VISIBLE_DEVICES можно перечислить индексы GPU, которые будут доступны. Исчерпывающее описание правил задания можно найти по ссылке http://acceleware.com/blog/cudavisibledevices-masking-gpus

In [1]:
import os
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

a = tf.constant(3.0, name='a')
b = tf.constant(4.0, name='b')    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run([a, b]))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15263809931967402913
]
[3.0, 4.0]


#### Вывод консоли хаба:

In [ ]:
# [name: "/device:CPU:0"
# device_type: "CPU"
# memory_limit: 268435456
# locality {
# }
# incarnation: 9352295143490226447
# ]
# Device mapping: no known devices.
# .............

# b: (Const): /job:localhost/replica:0/task:0/device:CPU:0
# 2018-01-31 15:03:12.782593: I tensorflow/core/common_runtime/placer.cc:874] b: (Const)/job:localhost/replica:0/task:0/device:CPU:0
# a: (Const): /job:localhost/replica:0/task:0/device:CPU:0
# 2018-01-31 15:03:12.782634: I tensorflow/core/common_runtime/placer.cc:874] a: (Const)/job:localhost/replica:0/task:0/device:CPU:0
# [3.0, 4.0]